In [ ]:
import cv2
import os

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
# Have to run this code as many face you have, for every face tag give different index number ie.1,2,3...n
face_id = input('\n Tag a new number for this face:-  ')
count = 0
#Input image forlder. must have single person photos
folder = r"F:\work\facer1\person1"
for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the indexed faces in dataset folder.. create a folder as "dataset" where your code file exist
        cv2.imwrite(r"ds\User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
#         cv2.imshow('image', img)
#         cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
# Dataset path
path = r"F:\work\facer1\ds"
recognizer = cv2.face.LBPHFaceRecognizer_create() 
detector = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml");

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') 
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

In [ ]:
import cv2
import numpy as np
import os 
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = cv2.data.haarcascades+"haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX
#iniciate id counter
id = 0
# Define min window size to be recognized as a face
minW = 0.1*3
minH = 0.1*4
# Give name with respect to ID... ie id-1  DHAMODAR... id-2  PRIYA... etc....
names = ['NONE', 'aishwarya' ,'katrina','samantha','shruti'] 
# Get the image and tag name

img = cv2.imread(r"F:\work\facer1\text1.jpg")
img = cv2.resize(img,(600,600))
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale( 
    gray,
    scaleFactor = 1.2,
    minNeighbors = 5,
    minSize = (int(minW), int(minH)))
for(x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
    id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

    if (confidence < 100):
        id = names[id]
        confidence = "  {0}%".format(round(100 - confidence))
    else:
        id = "unknown"
        confidence = "  {0}%".format(round(100 - confidence))

    cv2.putText(
                img, 
                str(id), 
                (x+5,y-5), 
                font, 
                1, 
                (255,255,255), 
                2
               )
    cv2.putText(
                img, 
                str(confidence), 
                (x+5,y+h-5), 
                font, 
                1, 
                (255,255,0), 
                1
               )  

cv2.imshow('Tag Name',img) 
cv2.waitKey(0)
# Do a bit of cleanup
print("\n Hoooo! you made it....")

cv2.destroyAllWindows()